In [62]:
from pynq import Overlay
from pynq.lib import DMA
from pynq import MMIO
from pynq import Xlnk
import numpy as np

#########REMEMBER TO TEST NEW .V FILES, SHOULD RUN EVEN IF VALID IS NOT ASSERTED


ol = Overlay("/home/xilinx/rosetta/design_1.bit")
ol.download()
print(ol.ip_dict)
#dma_read_addr = ol.ip_dict["axi_dma_0"]["phys_addr"]

dma1 = ol.axi_dma_0
dma2 = ol.axi_dma_1
gpio=ol.axi_gpio_0
#gpio?
#dma3 = ol.axi_dma_2
#dma1?
#dma_s = MMIO(dma_read_addr, 128)



{'axi_dma_0': {'phys_addr': 1077936128, 'addr_range': 65536, 'type': 'xilinx.com:ip:axi_dma:7.1', 'state': None, 'interrupts': {}, 'gpio': {}, 'fullpath': 'axi_dma_0', 'driver': <class 'pynq.lib.dma.DMA'>}, 'axi_dma_1': {'phys_addr': 1078001664, 'addr_range': 65536, 'type': 'xilinx.com:ip:axi_dma:7.1', 'state': None, 'interrupts': {}, 'gpio': {}, 'fullpath': 'axi_dma_1', 'driver': <class 'pynq.lib.dma.DMA'>}, 'axi_gpio_0': {'phys_addr': 1092616192, 'addr_range': 65536, 'type': 'xilinx.com:ip:axi_gpio:2.0', 'state': None, 'interrupts': {}, 'gpio': {}, 'fullpath': 'axi_gpio_0', 'driver': <class 'pynq.lib.axigpio.AxiGPIO'>}}


In [63]:
from PIL import Image#, ImageFilter
from time import time
#Read image

start = time()
im = Image.open( '/home/xilinx/jupyter_notebooks/bee_waifu.png' ).resize((320,240))
im.save("image_resized.png", "PNG")
out_data = []
in_data = list(im.getdata())

print(im.size)

R, G, B = 0, 1, 2

for pixel in in_data:
    out_data.append(pixel[R])
    out_data.append(pixel[G])
    out_data.append(pixel[B])

numberOfData = im.size[0]*im.size[1]*3
numberOfDataBlur = (im.size[0]-2)*(im.size[1]-2)
#numberOfDataDilate = (im.size[0]-4)*(im.size[1]-4)
xlnk = Xlnk()
help(xlnk)
gray1_in_buffer = xlnk.cma_array(shape=(numberOfData,), dtype=np.uint32)
gray2_in_buffer = xlnk.cma_array(shape=(numberOfData,), dtype=np.uint32)


blur_out_buffer = xlnk.cma_array(shape=(numberOfDataBlur,), dtype=np.uint32)
dilate_out_buffer = xlnk.cma_array(shape=(numberOfDataBlur,), dtype=np.uint32)
blur_out_buffer?

#dilate_out_buffer = xlnk.cma_array(shape=(numberOfDataBlur,))


#for i in range(0, numberOfData):
    
#    gray2_in_buffer[i] = out_data[i]
    #out1_buffer[i] = 0

for i in range(0, numberOfData):#numberOfData//3):
    gray2_in_buffer[i] = out_data[i]
    gray1_in_buffer[i] = out_data[i]

end = time()
print("Before dma.send took: ",end-start)


dma1.sendchannel.transfer(gray1_in_buffer)
dma1.recvchannel.transfer(blur_out_buffer)


dma1.sendchannel.wait()
dma1.recvchannel.wait()

dma2.sendchannel.transfer(gray2_in_buffer)
dma2.recvchannel.transfer(dilate_out_buffer)


#dma1.sendchannel.wait()
  
start = time()

blurim = Image.new("RGB", (im.size[0]-2, im.size[1]-2))
dilateim = Image.new("RGB", (im.size[0]-2, im.size[1]-2))
dilate_data = []
blur_data = []
for i in range(0, numberOfDataBlur):
    channel_blur = blur_out_buffer[i] #+100 
    channel_dilate = dilate_out_buffer[i]
    #pixel = (out1_buffer[i]+70, out1_buffer[i]+70, out1_buffer[i]+70)
    #data.append((out1_buffer[i]+50, out1_buffer[i]+50, out1_buffer[i]+50))
    blur_data.append((channel_blur, channel_blur, channel_blur))
    dilate_data.append((channel_dilate, channel_dilate, channel_dilate))
    if (i>((im.size[0]-2)*(im.size[1]-2)-20)):
        print((channel_blur, channel_blur, channel_blur))
#print("==============================================")
#for i in range(0, numberOfDataBlur):
#    channel_gray = dilate_out_buffer[i] #+100 
    #pixel = (out1_buffer[i]+70, out1_buffer[i]+70, out1_buffer[i]+70)
    #data.append((out1_buffer[i]+50, out1_buffer[i]+50, out1_buffer[i]+50))
    
#    dilate_data.append((channel_gray, channel_gray, channel_gray))
    #if (i<320):
    #    print((channel_gray,channel_gray,channel_gray))
  


    
    
blurim.putdata(blur_data)
blurim.save('image_blurred.png', 'PNG')

dilateim.putdata(dilate_data)
dilateim.save('image_dilated.png', 'PNG')

end = time()
print("Rebuilding image took: ", end-start)

gray1_in_buffer.close()
gray2_in_buffer.close()
blur_out_buffer.close()
dilate_out_buffer.close()

(320, 240)
Help on Xlnk in module pynq.xlnk object:

class Xlnk(builtins.object)
 |  Class to enable CMA memory management.
 |  
 |  The CMA state maintained by this class is local to the 
 |  application except for the `CMA Memory Available` attribute
 |  which is global across all the applications.
 |  
 |  Attributes
 |  ----------
 |  bufmap : dict
 |      Mapping of allocated memory to the buffer sizes in bytes.
 |  
 |  ffi : cffi instance
 |      Shared-object interface for the compiled CMA shared object
 |  
 |  Note
 |  ----
 |  If this class is parsed on an unsupported architecture it will issue
 |  a warning and leave the class variable libxlnk undefined
 |  
 |  Methods defined here:
 |  
 |  __del__(self)
 |      Destructor for the current Xlnk object.
 |      
 |      Frees up all the memory which was allocated through current object.
 |      
 |      Returns
 |      -------
 |      None
 |  
 |  __init__(self)
 |      Initialize new Xlnk object.
 |      
 |      Returns


Before dma.send took:  2.37827205657959
(235, 235, 235)
(235, 235, 235)
(235, 235, 235)
(235, 235, 235)
(235, 235, 235)
(235, 235, 235)
(235, 235, 235)
(235, 235, 235)
(235, 235, 235)
(235, 235, 235)
(235, 235, 235)
(235, 235, 235)
(235, 235, 235)
(235, 235, 235)
(235, 235, 235)
(235, 235, 235)
(235, 235, 235)
(235, 235, 235)
(0, 0, 0)
Rebuilding image took:  1.8231201171875


In [30]:
xlnk.xlnk_reset()
#dma1.sendchannel.transfer(in_buffer)
#dma1.recvchannel.transfer(out1_buffer)


#dma.sendchannel.transfer(in_buffer)
#dma2.recvchannel.transfer(out2_buffer)
#dma3.recvchannel.transfer(out3_buffer)
#dma.sendchannel.wait()
#dma.recvchannel.wait()



In [9]:
gpio.channel1()

#ps_switch = GPIO(GPIO.get_gpio_pin(1), 'in')


#ps_switch.read(1)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63


In [15]:


#from scipy import ndimage
#from scipy import misc
#f = misc.face()
#misc.imsave('face.png', f) # uses the Image module (PIL)

#import matplotlib.pyplot as plt
#plt.imshow(f)
#plt.show()

#image = open("/home/xilinx/rosetta/image.jpg", "rb")
#while(True):
 #   contents = image.readline()
  #  print(contents)
   # if contents == '':
    #    break
#image.close()

In [13]:
image.close()

In [41]:
320*240

76800

In [16]:
0.077847*0  + 0.123317*1  + 0.077847*2 + 0.123317*10 + 0.195346*11 + 0.123317*12 + 0.077847*20 + 0.123317*21 + 0.077847*22 

11.000022

In [17]:
  0.077847*1  + 0.123317*2  + 0.123317*3+ 0.123317*11 + 0.195346*12 + 0.123317*13+ 0.077847*21 + 0.123317*22 + 0.077847*23 

12.136434